# Проверка сервиса ASR

In [7]:
import requests
url = "http://localhost:8000/process-audio/"

# Путь к тестовому WAV-файлу
audio_file_path = "test_audio.wav"

# Путь к тестовому WAV-файлу
audio_file_path = "C:\\_tmp\\E4.wav"

# Список тестовых фраз
phrases = "hello world;test phrase;how are you;Открыть файл"

# Отправка POST-запроса
with open(audio_file_path, "rb") as audio_file:
    response = requests.post(
        url,
        files={"file": ("test_audio.wav", audio_file, "audio/wav")},
        data={"phrases": phrases}
    )

# Вывод результата
if response.status_code == 200:
    print("Response JSON:", response.json())
else:
    print(f"Error: {response.status_code}", response.text)


Response JSON: {'recognized_text': '[музыка'}


In [3]:
import torch
#Определим устройство для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from transformers import pipeline
from scipy.io import wavfile
import numpy as np
#транскрибатор
transcriber = pipeline("automatic-speech-recognition", model="lorenzoncina/whisper-small-ru")




samplerate, data = wavfile.read(audio_file_path)

if data.ndim > 1:
    data = data.mean(axis=1)
    
data = data.astype(np.float32)
data /= np.max(np.abs(data))
question = transcriber({"sampling_rate": samplerate, "raw": data})["text"]  
question


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
c:\Users\rvv19\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


'Добрый вечер, уважаемые Дежурной бригады, испытатели Е четыре сегодня двадцать первое апреля двадцать один сорок девять на среднем меня отличное самочувствие'

# Проверка сервиса TTS

In [5]:
# URL сервиса FastAPI
port = '8080'
speaker = 'baya'  # aidar, baya, kseniya, xenia, eugene
words = "прием прием начинаем ежедневную планерку"
sample_rate = 8000# 8000, 24000, 48000
# URL сервиса FastAPI

url = f"http://localhost:{port}/get_audio_file/?words=\"{words}\"&speaker={speaker}&sample_rate={sample_rate}"
response = requests.get(url)
# Проверяем, успешно ли прошел запрос
if response.status_code == 200:
    # Сохраняем полученный файл
    with open("processed_sound.wav", "wb") as output_sound:
        output_sound.write(response.content)
    print("Сгенерированный звук сохранен как processed_sound.wav")
else:
    print("Ошибка при отправке запроса:", response.status_code)


Сгенерированный звук сохранен как processed_sound.wav


In [6]:
import IPython
IPython.display.Audio("processed_sound.wav")